Ag Census Page: https://quickstats.nass.usda.gov/api

Use https://quickstats.nass.usda.gov/ to explore values...

In [2]:
#Imports
import pandas as pd
import numpy as np
import requests, os
import urllib

In [3]:
#Read in Ag Census Key
df = pd.read_csv('{}\\APIkeys.csv'.format(os.environ['localappdata']))
AgKey = df.loc[df.Service == 'NASS','Key'].values[0]

In [7]:
#Gets poultry data
for commodity in ['HOGS','CATTLE','CHICKENS']:
    apiURL = 'https://quickstats.nass.usda.gov/api/api_GET'
    params = {'key':AgKey,#   
              'source_desc':'CENSUS',
              'commodity_desc':commodity,
              'domain_desc':'TOTAL',
              'unit_desc':'HEAD',
              'state_fips_code':'37',
              'year':'2017',
              'agg_level_desc':'COUNTY'
             }
    response_raw = requests.get(apiURL,params)
    response_json = response_raw.json()

    #Report if the request was errant
    if(response_raw.status_code == 400):
        print("Broken URL for {}\n{}".format(commodity,response_raw.url))
        continue
        
    #Otherwise, keep going
    df = pd.DataFrame(requests.get(apiURL,params).json()['data'])

    #Fix values in "Values" column: 
    # Remove commas
    df['Value'] = df.Value.apply(lambda x: x.replace(",",""))
    #Set non numeric value to nan
    df['Value'] = df.Value.apply(lambda x: x if x.isnumeric() else np.nan)
    #Convert type to float
    df['Value'] = df.Value.astype(float)

    print("{} records returned for {}".format(len(df),commodity))
    
    #Pivot
    dfPivot = df.pivot_table(index = 'county_name',columns=['short_desc'],values='Value')
    dfPivot.to_csv('../data/processed/{}_2017.csv'.format(commodity))

243 records returned for HOGS
801 records returned for CATTLE
787 records returned for CHICKENS
